In [13]:
import os

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

In [3]:
datasets, info = tfds.load(
    name='mnist', with_info=True, as_supervised=True)
train, test = datasets['train'], datasets['test']

In [5]:
strategy = tf.distribute.MirroredStrategy()
strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


1

In [6]:
n_train = info.splits['train'].num_examples
n_test = info.splits['test'].num_examples

In [7]:
BUFF = 10_000
REPLICA_BATCH = 64
BATCH = REPLICA_BATCH * strategy.num_replicas_in_sync

In [8]:
def scale(img, label):
    img = tf.cast(img, tf.float32)
    img /= 255
    return img, label

In [10]:
train = train.map(scale).cache().shuffle(BUFF).batch(BATCH)
test = test.map(scale).cache().shuffle(BUFF).batch(BATCH)

In [12]:
with strategy.scope():
    mod = Sequential([
        Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10)])

In [14]:
mod.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
            optimizer=Adam(),
            metrics=['accuracy'])

In [15]:
mod.fit(train, epochs=12)

Epoch 1/12
938/938 [==============================] - 9s 9ms/step - loss: 0.3878 - accuracy: 0.8908
Epoch 2/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0718 - accuracy: 0.9788
Epoch 3/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0477 - accuracy: 0.9862
Epoch 4/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0341 - accuracy: 0.9898
Epoch 5/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0270 - accuracy: 0.9918
Epoch 6/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0203 - accuracy: 0.9938
Epoch 7/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0155 - accuracy: 0.9953
Epoch 8/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0151 - accuracy: 0.9951
Epoch 9/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0093 - accuracy: 0.9972
Epoch 10/12
938/938 [==============================] - 8s 9ms/step - loss: 0.0073 - accuracy: 0.9978